# Created aggregated checklist

In [1]:
import pandas as pd
from collections import OrderedDict

In [2]:
data = pd.read_table('../data/processed/checklist.tsv', dtype=object)

In [3]:
data.head()

,index,nameMatchValidation,taxonID,datasetName,taxonRank,scientificName,verbatimScientificName,synonym,kingdom,phylum,...,notes,manual_acceptedKey,gbifapi_usageKey,gbifapi_scientificName,gbifapi_canonicalName,gbifapi_status,gbifapi_rank,gbifapi_matchType,gbifapi_acceptedKey,gbifapi_acceptedScientificName
0,0,ok,NaN,fishes,species,Acipenser baerii,Acipenser baeri,NaN,Animalia,NaN,...,NaN,NaN,4287131,"Acipenser baerii Brandt, 1869",Acipenser baerii,ACCEPTED,SPECIES,EXACT,4287131,"Acipenser baerii Brandt, 1869"
1,1,ok,NaN,fishes,species,Acipenser gueldenstaedtii,Acipenser guldenstaedti,NaN,Animalia,NaN,...,NaN,NaN,2402129,"Acipenser gueldenstaedtii Brandt & Ratzeburg, ...",Acipenser gueldenstaedtii,ACCEPTED,SPECIES,EXACT,2402129,"Acipenser gueldenstaedtii Brandt & Ratzeburg, ..."
2,2,ok,NaN,fishes,species,Acipenser ruthenus,Acipenser ruthenus,NaN,Animalia,NaN,...,NaN,NaN,2402168,"Acipenser ruthenus Linnaeus, 1758",Acipenser ruthenus,ACCEPTED,SPECIES,EXACT,2402168,"Acipenser ruthenus Linnaeus, 1758"
3,3,ok,NaN,fishes,species,Ameiurus nebulosus,Ameiurus nebulosus,NaN,Animalia,NaN,...,NaN,NaN,2340989,"Ameiurus nebulosus (Lesueur, 1819)",Ameiurus nebulosus,ACCEPTED,SPECIES,EXACT,2340989,"Ameiurus nebulosus (Lesueur, 1819)"
4,4,ok: SYNONYM verified,NaN,fishes,species,Aspius aspius,Aspius aspius,NaN,Animalia,NaN,...,NaN,NaN,2360181,"Aspius aspius (Linnaeus, 1758)",Aspius aspius,SYNONYM,SPECIES,EXACT,5851603,"Leuciscus aspius (Linnaeus, 1758)"


## Retrieve valid records only

Only records that contain ok, so no wrong matches, no matches or unverified synonyms.

In [4]:
validRecords = data[data['nameMatchValidation'].str.contains('ok')]

In [5]:
validRecords['index'].count()

2504

## Aggregate and sort on gbifapi_acceptedName & collect unique values for some columns

In [6]:
def get_unique_values(series):
    # dropna(): remove NaN values
    # tuple(): will collect all values of a series as a tuple
    # set(): will only keep (unordered) unique values
    # Returns a set
    
    # It would have been easier to just return a list(), so we can sort it too,
    # but then we get 'Function does not reduce' (http://stackoverflow.com/a/37955931)
    # so have to use tuple()
    return set(tuple(series.dropna()))

In [7]:
# Default dict syntax for aggregation does not preserve column order
# unique_values_per_column = {
#    'kingdom': lambda x: get_unique_values(x),
#    'invasionStage': lambda x: get_unique_values(x),
#    'datasetName': lambda x: get_unique_values(x),
#    'presenceBE': lambda x: get_unique_values(x),
#    'index': lambda x: get_unique_values(x)
# }

In [8]:
# OrderedDict syntax for aggregation does preserve column order
unique_values_per_column = OrderedDict([
    ('kingdom', lambda x: get_unique_values(x)),
    ('invasionStage', lambda x: get_unique_values(x)),
    ('datasetName', lambda x: get_unique_values(x)),
    ('presenceBE', lambda x: get_unique_values(x)),
    ('gbifapi_scientificName', lambda x: get_unique_values(x)),
    ('index', lambda x: get_unique_values(x))
])

In [9]:
aggregatedRecords = validRecords.groupby(['gbifapi_acceptedScientificName','gbifapi_acceptedKey']).agg(unique_values_per_column).reset_index()

In [10]:
aggregatedRecords = aggregatedRecords.sort_values(by='gbifapi_acceptedScientificName')

In [11]:
aggregatedRecords.head()

,gbifapi_acceptedScientificName,gbifapi_acceptedKey,kingdom,invasionStage,datasetName,presenceBE,gbifapi_scientificName,index
0,Abies alba Mill.,2685484,{Plantae},{introduced},{plants},{present},{Abies alba Mill.},{1778}
1,Abies grandis (Douglas ex D. Don) Lindl.,2685361,{Plantae},{introduced},{plants},{present},{Abies grandis (Douglas ex D. Don) Lindl.},{1779}
2,Abronia fragrans Nutt. ex Hook.,5384889,{Plantae},{introduced},{plants},{present},{Abronia fragrans Nutt.},{1659}
3,Abutilon theophrasti Medik.,3152614,{Plantae},{unknown},{plants},{present},{Abutilon theophrasti Medik.},{1610}
4,Acalypha indica L.,3056259,{Plantae},{introduced},{plants},{present},{Acalypha indica L.},{1221}


## Sort and concatenate unique values per column

In [12]:
def sort_and_concatenate(aSet, sortAs=str):
    # sortAs allows to sort more specific
    # {9, 200, 12} will be sorted as:
    # '12, 200, 9' with default str
    # '9, 12, 200' with int
    sortedList = sorted(aSet, key=sortAs)
    concatenatedList = ', '.join(str(i) for i in sortedList)
    return concatenatedList # a string

In [13]:
aggregatedRecords['kingdom'] = aggregatedRecords['kingdom'].apply(lambda x: sort_and_concatenate(x))
aggregatedRecords['invasionStage'] = aggregatedRecords['invasionStage'].apply(lambda x: sort_and_concatenate(x))
aggregatedRecords['datasetName'] = aggregatedRecords['datasetName'].apply(lambda x: sort_and_concatenate(x))
aggregatedRecords['presenceBE'] = aggregatedRecords['presenceBE'].apply(lambda x: sort_and_concatenate(x))
aggregatedRecords['gbifapi_scientificName'] = aggregatedRecords['gbifapi_scientificName'].apply(lambda x: sort_and_concatenate(x))
aggregatedRecords['index'] = aggregatedRecords['index'].apply(lambda x: sort_and_concatenate(x, int))

In [14]:
aggregatedRecords.head()

,gbifapi_acceptedScientificName,gbifapi_acceptedKey,kingdom,invasionStage,datasetName,presenceBE,gbifapi_scientificName,index
0,Abies alba Mill.,2685484,Plantae,introduced,plants,present,Abies alba Mill.,1778
1,Abies grandis (Douglas ex D. Don) Lindl.,2685361,Plantae,introduced,plants,present,Abies grandis (Douglas ex D. Don) Lindl.,1779
2,Abronia fragrans Nutt. ex Hook.,5384889,Plantae,introduced,plants,present,Abronia fragrans Nutt.,1659
3,Abutilon theophrasti Medik.,3152614,Plantae,unknown,plants,present,Abutilon theophrasti Medik.,1610
4,Acalypha indica L.,3056259,Plantae,introduced,plants,present,Acalypha indica L.,1221


## Write aggregated records to file

In [15]:
aggregatedRecords.to_csv('../data/processed/aggregated-checklist.tsv', sep='\t', index=False)